# Corruption

In [1]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import urllib2

In [2]:
#download html table

url = 'https://15mpedia.org/wiki/Lista_de_casos_de_corrupci%C3%B3n'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Caso = []
Ambito = []
Partido = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 4:
            
            Caso.append(cells[0].find(text=True).rstrip())
            
            if cells[1].find(text=True) == None: Ambito.append(np.nan)
            else: Ambito.append(cells[1].find(text=True).rstrip())
                
            if cells[2].find(text=True) == None: Partido.append(np.nan)
            else: Partido.append(cells[2].find(text=True).rstrip())
                
lista_corrupcion = pd.DataFrame()
lista_corrupcion['Caso'] = Caso
lista_corrupcion['Comunidad'] = Ambito
lista_corrupcion['Partido'] = Partido

lista_corrupcion['Partido'] = lista_corrupcion['Partido'].str.replace('Partido Popular', 'PP')
lista_corrupcion['Partido'] = lista_corrupcion['Partido'].str.replace(lista_corrupcion['Partido'].values[0], 'PSOE')


In [3]:
#download html table

url = 'https://es.wikipedia.org/wiki/Anexo:Casos_judiciales_relacionados_con_corrupci%C3%B3n_pol%C3%ADtica_en_Espa%C3%B1a'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Caso2 = []
Partido2 = []
Ambito2 = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 4:
            
            Caso2.append(cells[0].find(text=True).rstrip())
            
            if cells[1].find(text=True) == None: Partido2.append(np.nan)
            else: Partido2.append(cells[1].find(text=True).rstrip())
                
            if cells[3].find(text=True) == None: Ambito2.append(np.nan)
            else: Ambito2.append(cells[3].find(text=True).rstrip())
                
lista_corrupcion2 = pd.DataFrame()
lista_corrupcion2['Caso'] = Caso2
lista_corrupcion2['Comunidad'] = Ambito2
lista_corrupcion2['Partido'] = Partido2


In [4]:
#concat the two tables
frames = [lista_corrupcion, lista_corrupcion2]
m_frame = pd.concat(frames)

#remove duplicates based on column Caso
m_frame = m_frame.drop_duplicates('Caso')

#remove rows with nans either in 'Comunidad' or 'Partido'
m_frame = m_frame.dropna()

#show corruption cases for instance in Galicia
m_frame[m_frame['Comunidad'] == 'Galicia']

,Caso,Comunidad,Partido
9,Caso Baltar,Galicia,PP
21,Caso Campeón,Galicia,PSOE
81,Caso Pokémon,Galicia,PP
5,Caso Arena,Galicia,PP
21,Caso Caballo de Troya,Galicia,PP
30,Caso Carioca,Galicia,PP
100,Caso Patos,Galicia,PP
119,Caso Relámpago,Galicia,PSOE
144,Caso Zeta,Galicia,PP


In [5]:
#replacement of some ccaa names to make them consistent

m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('IBIZA', 'Baleares')
m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('puebla de almenara', 'Castilla-La Mancha')
m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('Roquetas del Mar', 
                                                        list(set(m_frame['Comunidad'].values))[1] ) #andalucía

m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('Burgos', list(set(m_frame['Comunidad'].values))[9])
                                                                                        #Castilla y León
    
m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('Barcelona',list(set(m_frame['Comunidad'].values))[0])

m_frame['Comunidad'] = m_frame['Comunidad'].str.replace('Nacional', 'Comunidad de Madrid')
m_frame['Comunidad'] = m_frame['Comunidad'].str.replace(list(set(m_frame['Comunidad']))[19], 'Murcia')

#drop rows that do not belong to any ccaa
m_frame = m_frame[m_frame.Comunidad != 'Ministerio de Fomento.Adif']
m_frame = m_frame[m_frame.Comunidad != 'Varias provincias']
m_frame = m_frame[m_frame.Comunidad != 'Madrid']
m_frame = m_frame[m_frame.Comunidad != 'Valencia']
m_frame = m_frame[m_frame.Comunidad != list(set(m_frame['Comunidad']))[1]]
m_frame = m_frame[m_frame.Comunidad != list(set(m_frame['Comunidad']))[1]]
m_frame = m_frame[m_frame.Comunidad != list(set(m_frame['Comunidad']))[11]]

#reindex dataframe
m_frame.index = np.arange(0, len(m_frame))

#drop some meaningless rows (not associated to political parties)
m_frame = m_frame.drop(m_frame.index[[18, 17, 25]])

#add external data for communities: La Rioja, Cantabria, Aragón

df = pd.DataFrame([['Caso PlaZa','Aragon','PSOE'], ['Operacion Molinos','Aragon', 'Partido Aragones (PAR)' ]], 
                                                                          columns=['Caso','Comunidad', 'Partido'])
frames = [m_frame, df]
m_frame = pd.concat(frames)

In [6]:
#get latitude and longitudes of each capital communities
import re

url = 'https://es.wikipedia.org/wiki/Anexo:Capitales_de_comunidades_aut%C3%B3nomas_de_Espa%C3%B1a'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Comunidad = []
Capital = []
Coordenadas = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 6:
            
            Comunidad.append(re.findall('title="(.*?)">', str(cells[0]))[0].decode('utf-8'))
            Capital.append(re.findall('title="(.*?)">', str(cells[1]))[0])
            Coordenadas.append(re.findall(r"\d+\.\d+", str(cells[5]))[0:2])
            
Comunidad[3] = Comunidad[3].replace(Comunidad[3], 'Baleares')

Coordenadas_updated = []
for i in range(len(Coordenadas)):
    if Capital[i] != 'Palma de Mallorca' and Capital[i] != 'Barcelona':
        lon = float(Coordenadas[i][0])
        lat = float(Coordenadas[i][1]) * -1
        Coordenadas_updated.append([lon,lat])
    else:
        Coordenadas_updated.append([float(Coordenadas[i][0]),float(Coordenadas[i][1])])
        
ccaa_coord = pd.DataFrame()
ccaa_coord['Comunidad'] = Comunidad
ccaa_coord['Capital'] = Capital
ccaa_coord['Coordenadas'] = Coordenadas_updated

#make community names consistent between the 2 datasets to merge them

ccaa_coord['Comunidad'] = ccaa_coord['Comunidad'].str.replace(ccaa_coord.Comunidad.values[1], 'Aragon')
ccaa_coord['Comunidad'] = ccaa_coord['Comunidad'].str.replace(ccaa_coord.Comunidad.values[15], 'Murcia')

ccaa_coord.head()

,Comunidad,Capital,Coordenadas
0,Andalucía,Sevilla,"[37.367, -5.983]"
1,Aragon,Zaragoza,"[41.65, -0.883]"
2,Asturias,Oviedo,"[43.36, -5.845]"
3,Baleares,Palma de Mallorca,"[39.567, 2.65]"
4,Canarias,Santa Cruz de Tenerife,"[28.15, -15.417]"


In [7]:
df = m_frame.merge(ccaa_coord,  how='left')
cols = df.columns.tolist()
# reorder columns
cols = [cols[0]] + [cols[2]] + [cols[1]] + [cols[3]] + [cols[4]]
# "commit" the reordering
df = df[cols]
df = df.dropna()
df[3:7]

,Caso,Partido,Comunidad,Capital,Coordenadas
3,Caso Andratx,PP,Baleares,Palma de Mallorca,"[39.567, 2.65]"
4,Caso Arcos,PP,Andalucía,Sevilla,"[37.367, -5.983]"
5,Caso Arona,Coalición Canaria,Canarias,Santa Cruz de Tenerife,"[28.15, -15.417]"
6,Caso Arroyo,PP,Castilla y León,Valladolid,"[41.633, -4.717]"


In [8]:
dic_parties = {party:index for index, party in enumerate(list(set(df['Partido'])))}

dic_parties[dic_parties.keys()[0]] = 'CiU'
dic_parties[dic_parties.keys()[1]] = 'PP'
dic_parties[dic_parties.keys()[2]] = 'UPN'
dic_parties[dic_parties.keys()[3]] = 'UCOR'
dic_parties[dic_parties.keys()[4]] = 'PSOE'
dic_parties[dic_parties.keys()[5]] = 'UDC'
dic_parties[dic_parties.keys()[6]] = 'CC'
dic_parties[dic_parties.keys()[7]] = 'CDC'
dic_parties[dic_parties.keys()[8]] = 'PNL-NC'
dic_parties[dic_parties.keys()[9]] = 'PSC-PSOE'
dic_parties[dic_parties.keys()[10]] = 'PSOE'
dic_parties[dic_parties.keys()[11]] = 'PNV'
dic_parties[dic_parties.keys()[12]] = 'GIL'
dic_parties[dic_parties.keys()[13]] = 'PSIB-PSOE'
dic_parties[dic_parties.keys()[14]] = 'ERC'
dic_parties[dic_parties.keys()[15]] = 'UM'
dic_parties[dic_parties.keys()[16]] = 'PSE-EE'
dic_parties[dic_parties.keys()[17]] = 'PP'

parties = [items for index, items in df['Partido'].iteritems()]
party_acronyms = [w.replace(w, dic_parties[w]) for w in parties]

df['Partido'] = party_acronyms
df.head()

,Caso,Partido,Comunidad,Capital,Coordenadas
0,Caso AVE,PSOE,Comunidad de Madrid,Madrid,"[40.433, -3.683]"
1,Caso Acuamed,PP,Comunidad de Madrid,Madrid,"[40.433, -3.683]"
2,Caso Almenas,PSOE,Castilla-La Mancha,Toledo,"[23.5, -30.4]"
3,Caso Andratx,PP,Baleares,Palma de Mallorca,"[39.567, 2.65]"
4,Caso Arcos,PP,Andalucía,Sevilla,"[37.367, -5.983]"


In [9]:
# create json file to read in javascript
import json
from pprintpp import pprint as pp

ccaa_list = []
# write df as JSON, bring into appropriate format
obj = {'objects': list()}
for index, ccaa in df['Comunidad'].iteritems():
    df_ccaa = df[df['Comunidad'] == ccaa]
    if ccaa not in ccaa_list:
        obj['objects'].append({'circle': {
                    'coordenadas': df_ccaa.Coordenadas.values[0], 
                    'comunidad': df_ccaa.Comunidad.values[0],
                    'capital': df_ccaa.Capital.values[0],
                    'caso_partido': [[df_ccaa.Caso.values[index], df_ccaa.Partido.values[index]]
                                                              for index in range(len(df_ccaa.Partido.values))]
                }})
        
        ccaa_list.append(ccaa)

#with open('../data/casos_comunidad.json', 'w') as data_file:    
#    json.dump(obj, data_file)
    
pp(obj['objects'][3:4])


[
    {
        'circle': {
            'capital': 'Sevilla',
            'caso_partido': [
                [u'Caso Arcos', u'PP'],
                [u'Caso Astapa', u'PSOE'],
                [u'Caso Guerra', u'PSOE'],
                [u'Caso La Fabriquilla', u'PP'],
                [u'Caso Malaya', u'UCOR'],
                [u'Caso Matsa', u'PSOE'],
                [u'Caso Poniente', u'PSOE'],
                [u'Caso Sanlúcar', u'PSOE'],
                [u'Caso Terán', u'PSOE'],
                [u'Caso Troya', u'PP'],
                [u'Caso de la rotonda de Armilla', u'PSOE'],
                [u'Caso de los ERE en Andalucía', u'PSOE'],
                [u'Caso de los cursos de formación', u'PSOE'],
                [u'Caso nevada', u'PSOE'],
                [u'Caso vertidos Armilla', u'PSOE'],
                [u'Operación EDUende', u'PSOE'],
                [u'Caso Atarfe', u'PSOE'],
                [u'Caso Dolores Reyes', u'PSOE'],
                [u'Caso Saqueo I', u'GIL'],
          

In [10]:
#count most corrupted political party per community
from collections import Counter

color = {}
color['PP'] = '#00008B'
color['PSOE'] = '#DC143C'
color['PSC-PSOE'] = '#A0522D'
color['PNV'] = '#2E8B57'
color['UPN'] = '#DAA520' #Unión del Pueblo Navarro
 
data = {'data': list()}
for i in range(len(obj['objects'])):
    partidos = []
    for j in range(len(obj['objects'][i]['circle']['caso_partido'])):
        partidos.append(obj['objects'][i]['circle']['caso_partido'][j][1])
    c = Counter(partidos)
    ccaa = obj['objects'][i]['circle']['comunidad']
    data['data'].append({ccaa:{
            'partido corrupto': c.most_common(1)[0][0],
            'casos': len(partidos),
            'color comunidad': color[c.most_common(1)[0][0]]}})
    
data['data'].append({'otro':{
            'casos': 0,
            'color comunidad': '#FA8072',
            'partido corrupto': 'NC'}})
        
pp(data['data'][1:4])

[
    {
        u'Castilla-La Mancha': {
            'casos': 3,
            'color comunidad': '#DC143C',
            'partido corrupto': u'PSOE',
        },
    },
    {
        u'Baleares': {
            'casos': 13,
            'color comunidad': '#00008B',
            'partido corrupto': u'PP',
        },
    },
    {
        u'Andalucía': {
            'casos': 20,
            'color comunidad': '#DC143C',
            'partido corrupto': u'PSOE',
        },
    },
]


In [11]:
#load spain-community json file to modify it (insert some data for visualizing it)

with open('../data/spain-communities.geojson') as data_file:    
    dt = json.load(data_file)

In [12]:
#create a dict 0 is key, castilla la mancha value. From castilla la mancha we add (number of cases, color)

dic = {}
ccaa2stats = [5,7,15,10,15,2,4,15,3,11,8,15,13,6,14,0,12,1,9]
dic = {index:item for index, item in enumerate(ccaa2stats)}


for i in range(19):
    ccaa = data['data'][ccaa2stats[i]].keys()[0]
    dt['features'][i]['properties']['color'] = data['data'][ccaa2stats[i]][ccaa]['color comunidad']
    dt['features'][i]['properties']['casos'] = data['data'][ccaa2stats[i]][ccaa]['casos']
    dt['features'][i]['properties']['partido'] = data['data'][ccaa2stats[i]][ccaa]['partido corrupto']
    
#write json to a file
with open('../data/spain-communities-final.geojson', 'w') as data_file:    
    json.dump(dt, data_file)


In [13]:
#2nd visualization

url = 'https://es.wikipedia.org/wiki/Anexo:Pol%C3%ADticos_de_Espa%C3%B1a_implicados_en_casos_judiciales'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Partido = []
Nombre = []
Grado_Implicacion = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 7:

            Partido.append(cells[2].find(text=True).rstrip())

            Nombre.append(cells[1].find(text=True).rstrip())

            if cells[5].find(text=True) == None: Grado_Implicacion.append(np.nan)
            else: Grado_Implicacion.append(cells[5].find(text=True).rstrip())

df_circles = pd.DataFrame()
df_circles['Partido'] = Partido
df_circles['Nombre'] = Nombre
df_circles['Grado Implicacion'] = Grado_Implicacion

df_circles = df_circles.dropna()
df_circles = df_circles[df_circles['Grado Implicacion'] == 'Imputado']

df_circles.head()

,Partido,Nombre,Grado Implicacion
0,PP,Álvaro Lapuerta,Imputado
1,PP,Ángel Acebes,Imputado
2,PP,Ángel Sanchís,Imputado
3,PP,Cristóbal Páez Vicedo,Imputado
4,PP,Jesús Merino Delgado,Imputado


In [14]:
df_ImputadosPorPartido = df_circles.groupby(['Partido']).count()
df_ImputadosPorPartido['Grado Implicacion']

Partido
CIU (CDC)                               18
Ciudadanos-Partido de la Ciudadanía      1
PP                                     138
PSC                                      8
PSOE                                    87
Name: Grado Implicacion, dtype: int64